# Reproduce Geometric SpeedUp Table

In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
#define custom function
def g_mean(x):
    a = np.log(x)
    return np.exp(a.mean())


def get_speedup(path):
    files = os.listdir(path)

    data = {"ga":{}, "ppo":{}}
    
    for file in files:
        if "ipynb" not in file and "DS_Store" not in file:
            name = file.split("_")[1]
            if "cpole" in path:
                size = file.split("-")[-1].replace(".pkl", "")
            else:
                size = file.split("-")[-1].split(".")[0]
            result = {}
            df = pd.read_pickle(path + file)
            result["Time to Train"] = round(df["avg_train_time"], 2)
            result["Time Std"] = round(np.std(df["train_time"]), 0)
            # result["Expected Return"] = round(df["mask_eval_reward"], 2)
            # result["Return Std"] = round(np.std(df["ep_reward"]), 0)
            # result["Safe Actions"] = round(df["mask_safe_rolls"], 2)
            data[name][size] = result
    
    s_stat = []
    if "cpole" in path:
        for i in ["1.5", "1.25", "1.0", "0.75", "0.5", "0.25"]:
            s_stat.append(round(data["ppo"][i]["Time to Train"]/data["ga"][i]["Time to Train"], 2))
    elif "frolake" in path:
        for i in ["8", "16", "32"]:
            s_stat.append(round(data["ppo"][i]["Time to Train"]/data["ga"][i]["Time to Train"], 2))
    else:
        for i in ["5", "50", "100"]:
            s_stat.append(round(data["ppo"][i]["Time to Train"]/data["ga"][i]["Time to Train"], 2))
    
    return round(g_mean(s_stat), 2)  

In [31]:
f = "run_time_assurance/knapsack/results/knapsack_ga_rta_seeded_results-5.pkl"

In [32]:
df = pd.read_pickle(f)

In [33]:
df

{'avg_train_time': 1.0512,
 'train_time': [1.0087580680847168,
  0.9981081485748291,
  1.026270866394043,
  0.9573078155517578,
  1.265521764755249],
 'mask_eval_reward': 36.0,
 'mask_safe_rolls': 100.0,
 'mask_v_total': 0}

In [ ]:
# create speedup dictionary to store values
s_vals = {
    "cpole":{},
    "frolake":{}, 
    "knapsack":{}
}

# start with action mask
am_path = "./action_masking/"
rta_path = "./run_time_assurance/"

walk = [rta_path, am_path]
all_envs = []

print("Geometric Mean xSpeedUp of SPGA vs. SRL")
print("--------------------------------------")
for i in s_vals.keys():
    for way in walk:
        path = way+i+"/results/"
        x = get_speedup(path)
        all_envs.append(x)
        print("{} - {} SpeedUp: {}".format(i, "AM" if way == am_path else "RTA", x))

print("All Envs: {}".format(round(g_mean(all_envs), 2)))